In [1]:
import os

import numpy as np

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [2]:
TEST = False

table = 'movement_model_inference_m4_a1_v1'

table = table if not TEST else f'{table}_test'
data = read_data_w_cache(
    f'select * from {table}'
)
data = data[data['_selected'].isin([True, False])]
data['log_prob'] = np.log(data['probability'])
data.head()

,_decision,_choice,_selected,mixed_layer_thickness,net_primary_production,water_heading,movement_heading,distance,origin_h3_index,next_h3_index,...,sin_time,tag_key,log_odds,odds,probability,experiment_name,run_id,_train,_partition,log_prob
0,5,3,False,10.528886,12.115724,-1.934817,-0.320495,84.158066,8412b2dffffffff,8412b25ffffffff,...,0.755750,229207,-1.885071,1.518183e-01,1.464378e-02,movement-model-m4-a1-v1,cb34734b6482840afabf073c3b014043256f6ee8388c13...,True,2,-4.223740
1,17,19,False,11.844339,9.233685,1.613477,3.141479,67.648079,8412b27ffffffff,8412949ffffffff,...,0.281733,229207,-6.840713,1.069341e-03,1.315347e-04,movement-model-m4-a1-v1,cb34734b6482840afabf073c3b014043256f6ee8388c13...,True,2,-8.936240
2,17,14,False,11.212727,12.960359,2.493804,0.356131,75.733185,8412b27ffffffff,8412949ffffffff,...,0.281733,229207,-2.167526,1.144605e-01,1.407926e-02,movement-model-m4-a1-v1,cb34734b6482840afabf073c3b014043256f6ee8388c13...,True,2,-4.263052
3,20,16,False,10.523017,22.055365,1.537851,0.000000,0.000000,8412b37ffffffff,841295dffffffff,...,0.281733,229207,-38.766376,1.458736e-17,1.322626e-18,movement-model-m4-a1-v1,cb34734b6482840afabf073c3b014043256f6ee8388c13...,True,2,-41.166912
4,5,2,False,10.347228,23.941929,-0.871481,-0.317017,42.011292,8412b2dffffffff,8412b25ffffffff,...,0.755750,229207,1.326427,3.767557e+00,3.634032e-01,movement-model-m4-a1-v1,cb34734b6482840afabf073c3b014043256f6ee8388c13...,True,2,-1.012242


In [3]:
final = (
    data[data['_selected']]
    .groupby(['_individual', '_train'])[['log_prob']]
    .mean().reset_index()
    .groupby('_train')[['log_prob']].mean()
)
final['GMP'] = np.exp(final['log_prob'])
final

,log_prob,GMP
_train,,
False,-1.747666,0.174180
True,-1.774778,0.169521


In [4]:
final = (
    data[data['_selected'] & (data['distance'] > 0)]
    .groupby(['_individual', '_train'])[['log_prob']]
    .mean().reset_index()
    .groupby('_train')[['log_prob']].mean()
)
final['GMP'] = np.exp(final['log_prob'])
final

,log_prob,GMP
_train,,
False,-1.747666,0.174180
True,-1.774778,0.169521
